# pix2pix

The original pix2pix technique was described in [Image-to-Image Translation with Conditional Adversarial Networks](https://arxiv.org/pdf/1611.07004v1.pdf) and after the initial paper was published there was an outpouring of implementations in a variety of languages and frameworks developed by the community.

We will be using Christopher Hesse's [pix2pix-tensorflow](https://github.com/affinelayer/pix2pix-tensorflow) implementation.

## Setup

In [1]:
print "test\n--"
!ls

test
--
datalab


In [2]:
import os
# download repository to this machine
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git
os.chdir('pix2pix-tensorflow')
print "--"
!ls

Cloning into 'pix2pix-tensorflow'...
remote: Counting objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 36.48 MiB/s, done.
Resolving deltas: 100% (106/106), done.
--
docker	docs  LICENSE.txt  pix2pix.py  README.md  server  tools


In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create GoogleDrive instance with authenticated GoogleAuth instance.
drive = GoogleDrive(gauth)

In [4]:
id = "1W9E2yOEMBbEcqGvldMVw2uY-bc9bZtqO"
filename = "cities.tar"
file = drive.CreateFile({'id': id})
file.GetContentFile(filename)
print('%s (%s) downloaded' % (file['title'], file['mimeType']))
!mkdir cities
!mkdir checkpoints
!tar -xf cities.tar
!rm cities/.DS_Store
!ls

cities.tar (application/x-tar) downlaoded
checkpoints  cities.tar  docs	      pix2pix.py  server
cities	     docker	 LICENSE.txt  README.md   tools


In [4]:
id = "1JiCOdgzwjnemy_fVF7es3HjTktCep1uQ"
filename = "ckpt_large.tar.gz"
file = drive.CreateFile({'id': id})
file.GetContentFile(filename)
print('%s (%s) downloaded' % (file['title'], file['mimeType']))
!tar -xf ckpt_large.tar.gz
!ls
!ls checkpoints

ckpt_large.tar.gz (application/x-tar) downlaoded
checkpoints	   docker  LICENSE.txt	README.md  tools
ckpt_large.tar.gz  docs    pix2pix.py	server
checkpoint				     model-148750.index
events.out.tfevents.1521344148.910fc596d154  model-148750.meta
graph.pbtxt				     options.json
model-148750.data-00000-of-00001


In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

try:
    from urllib.request import urlopen # python 3
except ImportError:
    from urllib2 import urlopen # python 2
import sys
import tarfile
import tempfile
import shutil

url = "http://hizalcelik.com/dev/cities_large.tar.gz"

with tempfile.TemporaryFile() as tmp:
    print("Downloading", url)
    shutil.copyfileobj(urlopen(url), tmp)
    print("Extracting")
    tmp.seek(0)
    if ".tar" in url or ".tgz" in url:
      tar = tarfile.open(fileobj=tmp)
      tar.extractall()
      tar.close()
    print("Done")

Extracting
Done


## Train the Model

In [7]:
!python pix2pix.py \
  --mode train \
  --output_dir checkpoints \
  --checkpoint checkpoints \
  --max_epochs 100 \
  --input_dir cities_large \
  --which_direction BtoA \
  --aspect_ratio 2

UnboundLocalError: ignored

In [0]:
!python pix2pix.py \
  --mode train \
  --output_dir checkpoints \
  --max_epochs 50 \
  --input_dir cities_large \
  --which_direction BtoA \
  --aspect_ratio 2

## Download Results

We can download out results and open them locally

In [12]:
!ls checkpoints
!du -hs checkpoints

checkpoint				     model-233750.index
events.out.tfevents.1521467551.5cbd78d1f6e5  model-233750.meta
graph.pbtxt				     options.json
model-233750.data-00000-of-00001
1.3G	checkpoints


In [0]:
!rm checkpoints/model-148750.data-00000-of-00001
!rm checkpoints/model-148750.index
!rm checkpoints/model-148750.meta
!rm checkpoints/events.out.tfevents.1521344148.910fc596d154

In [13]:
!tar -czvf ckpt_large2.tar.gz checkpoints
!du -hs ckpt_large2.tar.gz

checkpoints/
checkpoints/graph.pbtxt
checkpoints/model-233750.index
checkpoints/options.json
checkpoints/model-233750.meta
checkpoints/events.out.tfevents.1521467551.5cbd78d1f6e5
checkpoints/model-233750.data-00000-of-00001
checkpoints/checkpoint
861M	ckpt_large2.tar.gz


In [14]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create GoogleDrive instance with authenticated GoogleAuth instance.
drive = GoogleDrive(gauth)

filename = "ckpt_large2.tar.gz"
file = drive.CreateFile()
file.SetContentFile(filename)
file.Upload()
print('%s (%s) uploaded' % (file['title'], file['mimeType']))

ckpt_large2.tar.gz (application/x-tar) uploaded
